In [ ]:
%matplotlib inline
import xarray as xr
import torch
from uwnet.model import MLP

ds = xr.open_dataset("../data/processed/training.nc").isel(time=slice(0,120), z=slice(0,28))

mod = MLP.from_path("../models/29/1.pkl")

input = ds.isel(time=slice(0,120))

# this call computes FQTNN and FSLINN for each input snapshot
output = mod.call_with_xr(input)

# compute water budget diagnostics
PW_dot = ((output.FQTNN*input.layer_mass).sum('z') + (input.FQT*input.layer_mass).sum('z'))/1000*86400
pw = (output.QT*ds.layer_mass).sum('z')
pw_change = pw[-1]-pw[0]
dt = (pw.time[-1] -pw.time[0])
pw_change_mean = pw_change.mean('x')/1000/dt

In [ ]:
pw_change_mean.plot()
plt.title("PW Storage (mm/day)")

The observed simulation has little change in the zonal mean precipitable

In [ ]:
PW_dot.mean(['x', 'time']).plot()
plt.title("zonal+time mean FQTNN + FQT (mm/day)")

However, the sum of FQTNN and FQT has a substantial bias in the time and zonal mean. This probably explain the mean state bias in our simulations.